Hi People, This notebook is for complitely beginers. Here i took the dataset and by visualise this i found some good insights about this dataset. I also perform basic algorithms to find the output. If you find this intresting please upvote.

# Problem Statement:

The two datasets are related to red and white variants of the Portuguese "Vinho Verde" wine. For more details, consult the reference [Cortez et al., 2009]. These datasets can be viewed as classification or regression tasks. The classes are ordered and not balanced (e.g. there are much more normal wines than excellent or poor ones).

__Here our main task is to to find the the insights abour data and make a machine leraning model.__

This dataset is also available from the UCI machine learning repository, https://archive.ics.uci.edu/ml/datasets/wine+quality , I just shared it to kaggle for convenience. (If I am mistaken and the public license type disallowed me from doing so, I will take this down if requested.)

## About Dataset

__1.  fixed acidity:__ Most acids involved with wine or fixed or nonvolatile (do not evaporate readily).

__2.  volatile acidity:__ The amount of acetic acid in wine, which at too high of levels can lead to an unpleasant, vinegar taste.

__3.  citric acid:__ Found in small quantities, citric acid can add 'freshness' and flavor to wines.

__4.  residual sugar:__ The amount of sugar remaining after fermentation stops, it's rare to find wines with less than 1 gram/liter.

__5.  chlorides:__ The amount of salt in the wine.

__6.  free sulfur dioxide:__ The free form of SO2 exists in equilibrium between molecular SO2 (as a dissolved gas) and bisulfite ion; it prevents.

__7.  total sulfur dioxide:__ Amount of free and bound forms of S02; in low concentrations, SO2 is mostly undetectable in wine, but at free SO2.

__8.  density:__ The density of water is close to that of water depending on the percent alcohol and sugar content.

__9.  pH:__ Describes how acidic or basic a wine is on a scale from 0 (very acidic) to 14 (very basic); most wines are between 3-4.

__10. sulphates:__ A wine additive which can contribute to sulfur dioxide gas (S02) levels, wich acts as an antimicrobial.

__11. alcohol:__ The percent alcohol content of the wine.

__12. quality:__ Output variable (based on sensory data, score between 0 and 10).

# Implimentation

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import random
import warnings
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import seaborn as sns # visualization
import matplotlib.pyplot as plt # visualization
from matplotlib.colors import ListedColormap
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler # standardization
from sklearn.model_selection import train_test_split # Split dataset
from sklearn.neighbors import KNeighborsClassifier # KNN Model
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB # Naive Bayes Model
from sklearn.metrics import accuracy_score # Accuracy measurements
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# [01] Read Data

In [ ]:
wine = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

In [ ]:
wine.head()

In [ ]:
wine.info()

# Here we can see there is no null values in this dataset.

In [ ]:
wine.isnull().sum()

# No null values present in data frame.

In [ ]:
wine.shape

# Total data points are 1599

In [ ]:
wine.columns

# List of features we have in this dataset.

In [ ]:
wine['quality'].value_counts()

Here we can say most of wines are between 5 to 6 range which is average. 3 is the lowest qulity and 8 is the highest.

# [02] EDA

## [2.1] Corelation Matrix: Visualise the similarity between features

In [ ]:
corr = wine.corr()
corr

## [2.2] Heat Map using corelation metrix:

In [ ]:
# Generate a mask for the upper triangle
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

with sns.axes_style("white"):
    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(10, 8))
    ax = sns.heatmap(corr, cmap=cmap, mask=mask, vmax=.3, square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

Observation:

from here we notice that impact of **'residual sugar'**, **'free sulfur dioxide'** and **'ph'** on **'quality'** is neglisible. So we can drop them. Also we observer that similarity between **'alcohol'** and **'quality'** is maximum.

In [ ]:
wine.drop(["residual sugar",'free sulfur dioxide','pH'],axis = 1,inplace = True)
wine.head()

## [2.3] Univariate Analysis: On Alcohol

In [ ]:
sns.distplot(wine['alcohol'])
plt.show()

To analyze more,

Let's split **'alcohol'** into three parts such as, **low, median and high**.

In [ ]:
bins = [0, 10, 12, 15]
labels = ["low","median","high"]
wine['alcohol_label'] = pd.cut(wine['alcohol'], bins=bins, labels=labels)
wine.drop('alcohol',axis =1, inplace = True)
wine.head()

## [2.4] Univariate Analysis: On Quality

In [ ]:
sns.distplot(wine['quality'])
plt.show()

Similarly as alcohol, 

We can split **quality** also into three small chuncks such as **poor** i.e. from 0 to 4, **normal** i.e. from 5 or 6 and **excellent** i.e. from 7 to 10.

In [ ]:
bins = [0, 4, 6, 10]
labels = ["poor","normal","excellent"]
wine['quality_label'] = pd.cut(wine['quality'], bins=bins, labels=labels)
wine.drop('quality',axis =1, inplace = True)
wine.head()

## [2.5] Multivariate Analysis: Using Pairplots

In [ ]:
sns.pairplot(wine, hue="quality_label", palette="husl",diag_kind="kde")
plt.show()

Observation:

To betermine wine qulity, **volatile acidity and citric acid** can be our important features as the overlap of their distributions are vary less in compare to others.

## [2.6] Analysis On: Volatile Acidity and Citric Acid

In [ ]:
sns.FacetGrid(wine,hue='quality_label', height=5).map(sns.distplot,'volatile acidity').add_legend()
plt.show()

In [ ]:
sns.boxplot(x='quality_label',y='volatile acidity', data=wine)
plt.show()

Observation:

From this plot we can assure that, More than 50% of our excellent catagory wine have volatile acidity in between 0.3 to 0.5.

In [ ]:
sns.FacetGrid(wine,hue='quality_label', height=5).map(sns.distplot,'citric acid').add_legend()
plt.show()

In [ ]:
sns.boxplot(x='quality_label', y='citric acid',data=wine)
plt.show()

Observation:

Similary from this plot we can say, More than 50% of our excellent catagory wine have citric acid in between 0.3 to 0.5.

# [03] Model Creation

In [ ]:
wine.info()

In [ ]:
wine['alcohol_label'].value_counts()

In [ ]:
# Convert category values to numeric values by creating dummy featutes.
df_wine = pd.get_dummies(wine, columns=['alcohol_label'], drop_first=True)
df_wine.head()

In [ ]:
result = df_wine['quality_label']
df_wine.drop(['quality_label'], axis=1, inplace=True)
print(df_wine.shape, result.shape)

In [ ]:
# use 70% of the data for training and 30% for testing
X_train, X_test, Y_train, Y_test = train_test_split(df_wine, result, test_size=0.30, random_state=11)

When we deal with distance that case we need to standadise our data other wise we can use normal data. In our case for KNN we need scdardise data.

In [ ]:
# For KNN our dataset must have to standardised.
# No need standardised quality_label as it is the result column

scaler = StandardScaler()
scaler.fit(df_wine)
scaled_features = scaler.transform(df_wine)
df_wine_sc = pd.DataFrame(scaled_features, columns=df_wine.columns)

In [ ]:
# use 70% of the data for training and 30% for testing
X_train_sc, X_test_sc, y_train_sc, y_test_sc = train_test_split(df_wine_sc, result, test_size=0.30, random_state=11)

## [3.1] Train Model:

#### [3.1.2] For KNN

In [ ]:
# convert dataframe to nd numpy array
X_train_sc = X_train_sc.to_numpy()
y_train_sc = y_train_sc.to_numpy()

In [ ]:
def apply_knn(neigh, weight='uniform'):
    knn = KNeighborsClassifier(n_neighbors=neigh, weights=weight)
    knn.fit(X_train_sc,y_train_sc)
    pred_knn = knn.predict(X_test_sc)
    return pred_knn

In [ ]:
pred_knn_for_20 = apply_knn(20)
print('Accuracy of model at K=20 is', accuracy_score(y_test_sc, pred_knn_for_20))

Now it’s time to improve the model and find out the optimal k value.

#### [3.1.2] For LogisticRegression

In [ ]:
clf = LogisticRegression(random_state=0)
clf.fit(X_train_sc, y_train_sc)
pred_lr = clf.predict(X_test_sc)
print('Accuracy of model is', accuracy_score(y_test_sc, pred_lr))

## [3.2] Apply GridSearchCV to Optomise Parameters

#### [3.2.1] For KNN

In [ ]:
model = KNeighborsClassifier()

params = {'n_neighbors':list(range(1, 50, 2)), 'weights':['uniform', 'distance']}

gs = GridSearchCV(model, params, cv = 5, n_jobs=-1)

gs_results = gs.fit(X_train_sc, y_train_sc)

In [ ]:
print('Best Accuracy: ', gs_results.best_score_)
print('Best Parametrs: ', gs_results.best_params_)

**From above we can say that the best value k is 13.**

In [ ]:
best_k = 13
best_weights = 'distance'
pred_knn_for_Best_k = apply_knn(best_k, best_weights)
print('Accuracy of model at K=13 is ', accuracy_score(y_test_sc, pred_knn_for_Best_k))

#### [3.2.2] For LogisticRegression

In [ ]:
model = LogisticRegression(max_iter=10000)

params = [{'C': [10**-4, 10**-2, 10**0, 10**2, 10**4], 'penalty': ['l1', 'l2']}]

gs = GridSearchCV(model, params, cv=5, n_jobs=-1)

gs_results = gs.fit(X_train_sc, y_train_sc)

In [ ]:
print('Best Accuracy: ', gs_results.best_score_)
print('Best Parametrs: ', gs_results.best_params_)

**From above we can say that the best parameters are C = '1' & penalty = 'l2'.**

In [ ]:
lr = LogisticRegression(C=1, penalty='l2', random_state=0)

lr.fit(X_train_sc, y_train_sc)

pred_lr_for_Best_param = lr.predict(X_test_sc)

print('Accuracy of model at C = 1 and Penalty = l2 is', accuracy_score(y_test_sc, pred_lr_for_Best_param))

## [3.3] K fold Cross-Validation:

#### [3.3.1] for KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=13,weights='distance')
scores_knn = cross_val_score(knn, X_train_sc, y_train_sc, cv=10, scoring='accuracy')

In [ ]:
print(scores_knn)

In [ ]:
print(scores_knn.mean())

**From KNN model we can conclude that our model's best accuracy is approx 85%**

#### [3.3.2] for LogisticRegression

In [ ]:
lr = LogisticRegression(C=1, penalty='l2',random_state=0)
scores_lr = cross_val_score(lr, X_train_sc, y_train_sc, cv=10, scoring='accuracy')

In [ ]:
print(scores_lr)

In [ ]:
print(scores_lr.mean())

**From LR model we can conclude that our model's best accuracy is approx 84% which is a bit lower than KNN**

## [3.4] Confusion Matrice for KNN

In [ ]:
print(pd.DataFrame(y_test_sc)['quality_label'].value_counts())

In [ ]:
cm = confusion_matrix(y_test_sc, pred_knn_for_Best_k)

In [ ]:
names = ["excellent","normal","poor"]
print(pd.DataFrame(cm, index=names, columns=names))

## [3.4] Confusion Matrice for LR

In [ ]:
cm = confusion_matrix(y_test_sc, pred_lr_for_Best_param)

In [ ]:
names = ["excellent","normal","poor"]
print(pd.DataFrame(cm, index=names, columns=names))

# [06] Conclusion:

However, what we should not ignore is that the wine quality classes are ordered and not balanced. There are much more normal wines than excellent or poor ones. Consequently, as we can see from the confusion matrix above, the predictive ability of the model for normal class(accuracy rate = 0.966) is much better than poor(accuracy rate = 0) and excellent class(accuracy rate = 0.265).

For the next step, we should try to find out methods to improve the model performance for unbalanced dataset.

## Thank for going through this notebook. If you find this notebook useful then please upvote.